# TP : canal de diffusion anonyme et génération de secret

Ce TP va consister à […].

In [20]:
from datetime import datetime
import random

## Partie 1

Dans cette première partie, on .

In [21]:
DELAI_MIN_MS = 1
DELAI_MAX_MS = 10


class Canal:
    file_messages = []

    def poster_message_anonyme(self, message: str):
        self.file_messages.append({
            "message": message,
            "date": datetime.now(),
        })

    def recuperer(self, debut: datetime, fin: datetime):
        return [message for message in file_messages if debut <= message["date"] <= fin]


def generer_secret(canal: Canal, complice_a: str, complice_b: str, duree_ms: int):
    debut = datetime.now()
    messages_complice_a = []
    messages_complice_b = []

    # Tant que le délai n'est pas écoulé
    while (datetime.now() - debut).microseconds < duree_ms * 1_000:
        # Génère un message contenant aléatoirement le nom du complice A ou du complice B
        message = {
            "message": complice_a if random.random() < 0.5 else complice_b,
            "date": datetime.now(),
        }

        # Enregistre le message au nom du complice A ou du complice B avec une chance équiprobable
        if random.random() < 0.5:
            messages_complice_a.append(message)
        else:
            messages_complice_b.append(message)

        # Et l'envoie sur le canal
        canal.poster_message_anonyme(message["message"])

        # Laisse un délai aléatoire s'éoculer avant le prochain message
        sleep(random.randrange(DELAI_MIN_MS, DELAI_MAX_MS))

    return (messages_complice_a, messages_complice_b)


def extraire_secret(canal: Canal, messages_complice: list[dict], debut: datetime, duree_ms: int):
    liste_messages = canal.recuperer(debut, debut + duree)
    return [0 if message in messages_complice and message["message"] == complice else 1 for message in liste_messages]

## Tests

[TODO les tests unitaires]